In [ ]:
import preprocessor as p
import numpy as np 
import pandas as pd 
import emoji
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
#from keras.layers.normalization import Batch Normalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm import tqdm

In [ ]:
data = pd.read_excel('output2.xlsx')

In [ ]:
data.head()

,LABEL,Tweet
0,terkejut,a nya bags banget bar ma baca tlisan nya canti...
1,sedih,aa gervan sayang lpa bat senym yah biar kam ga...
2,senang,aaa bangga banget hoonie u fd u fd u fd u fd u fd
3,jijik,aaaa iri iri iri dengki iri iri dengki u fd u ...
4,sedih,aaak suka banget feel habis nonton kayak the g...


In [ ]:
data.LABEL.value_counts()


terkejut    711
jijik       677
senang      316
marah       305
sedih       278
takut       139
Name: LABEL, dtype: int64

In [ ]:
sent_to_id  = {"terkejut":0, "jijik":1,"senang":2,"marah":3,"sedih":4,"takut":5}

In [ ]:
data["LABEL_id"] = data['LABEL'].map(sent_to_id)

In [ ]:
data

,LABEL,Tweet,LABEL_id
0,terkejut,a nya bags banget bar ma baca tlisan nya canti...,0
1,sedih,aa gervan sayang lpa bat senym yah biar kam ga...,4
2,senang,aaa bangga banget hoonie u fd u fd u fd u fd u fd,2
3,jijik,aaaa iri iri iri dengki iri iri dengki u fd u ...,1
4,sedih,aaak suka banget feel habis nonton kayak the g...,4
...,...,...,...
2421,jijik,yo tmb mak mak,1
2422,senang,yoon jisng asli ganteng drpda foto emang peson...,2
2423,sedih,ysfk nonton nyanyi dangdt yg cma jal goyang at...,4
2424,senang,zon lihat bajnya sajasekali-sekali pegang brng...,2


In [ ]:
sent_to_id1  = {"terkejut":"POSITIF", "jijik":"NEGATIF","senang":"POSITIF","marah":"NEGATIF","sedih":"NEGATIF","takut":"POSITIF"}

In [ ]:
data["SETIMEN"] = data['LABEL'].map(sent_to_id1)

In [ ]:
data

,LABEL,Tweet,LABEL_id,SETIMEN
0,terkejut,a nya bags banget bar ma baca tlisan nya canti...,0,NaN
1,sedih,aa gervan sayang lpa bat senym yah biar kam ga...,4,NEGATIF
2,senang,aaa bangga banget hoonie u fd u fd u fd u fd u fd,2,POSITIF
3,jijik,aaaa iri iri iri dengki iri iri dengki u fd u ...,1,NEGATIF
4,sedih,aaak suka banget feel habis nonton kayak the g...,4,NEGATIF
...,...,...,...,...
2421,jijik,yo tmb mak mak,1,NEGATIF
2422,senang,yoon jisng asli ganteng drpda foto emang peson...,2,POSITIF
2423,sedih,ysfk nonton nyanyi dangdt yg cma jal goyang at...,4,NEGATIF
2424,senang,zon lihat bajnya sajasekali-sekali pegang brng...,2,POSITIF


In [ ]:
sent_to_id2  = {"POSITIF":1, "NEGATIF":0}

In [ ]:
data["SETIMEN_id"] = data['SETIMEN'].map(sent_to_id2)

In [ ]:
data

,LABEL,Tweet,LABEL_id,SETIMEN,SETIMEN_id
0,terkejut,a nya bags banget bar ma baca tlisan nya canti...,0,POSITIF,1
1,sedih,aa gervan sayang lpa bat senym yah biar kam ga...,4,NEGATIF,0
2,senang,aaa bangga banget hoonie u fd u fd u fd u fd u fd,2,POSITIF,1
3,jijik,aaaa iri iri iri dengki iri iri dengki u fd u ...,1,NEGATIF,0
4,sedih,aaak suka banget feel habis nonton kayak the g...,4,NEGATIF,0
...,...,...,...,...,...
2421,jijik,yo tmb mak mak,1,NEGATIF,0
2422,senang,yoon jisng asli ganteng drpda foto emang peson...,2,POSITIF,1
2423,sedih,ysfk nonton nyanyi dangdt yg cma jal goyang at...,4,NEGATIF,0
2424,senang,zon lihat bajnya sajasekali-sekali pegang brng...,2,POSITIF,1


In [ ]:
data.drop(data.columns[[0,1,3]], axis = 1, inplace = True)

In [ ]:
data

,LABEL_id,SETIMEN_id
0,0,1
1,4,0
2,2,1
3,1,0
4,4,0
...,...,...
2421,1,0
2422,2,1
2423,4,0
2424,2,1


In [ ]:
import pandas as pd
import numpy as np
#visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#EDA
from collections import Counter
import pandas_profiling as pp
# data preprocessing
from sklearn.preprocessing import StandardScaler
# data splitting
from sklearn.model_selection import train_test_split
# data modeling
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
#ensembling
from mlxtend.classifier import StackingCVClassifier

In [ ]:
y = data["SETIMEN_id"]
X = data.drop('SETIMEN_id',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

In [ ]:
print(y_test.unique())
Counter(y_train)

[1 0]


Counter({0: 1010, 1: 930})

In [ ]:
X_test

,LABEL_id
2053,0
743,1
2297,0
1534,3
1840,1
...,...
1192,1
2265,5
254,1
1303,3


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
m6 = 'DecisionTreeClassifier'
dt = DecisionTreeClassifier(criterion = 'entropy',random_state=0,max_depth = 4)
dt.fit(X_train, y_train)
dt_predicted = dt.predict(X_test)
dt_conf_matrix = confusion_matrix(y_test, dt_predicted)
dt_acc_score = accuracy_score(y_test, dt_predicted)
print("confussion matrix")
print(dt_conf_matrix)
print("\n")
print("Accuracy of DecisionTreeClassifier:",dt_acc_score*100,'\n')
print(classification_report(y_test,dt_predicted))

confussion matrix
[[250   0]
 [  0 236]]


Accuracy of DecisionTreeClassifier: 100.0 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       250
           1       1.00      1.00      1.00       236

    accuracy                           1.00       486
   macro avg       1.00      1.00      1.00       486
weighted avg       1.00      1.00      1.00       486



In [ ]:
m4 = 'Extreme Gradient Boost'
xgb = XGBClassifier(learning_rate=0.01, n_estimators=25, max_depth=15,gamma=0.6, subsample=0.52,colsample_bytree=0.6,seed=27, 
                    reg_lambda=2, booster='dart', colsample_bylevel=0.6, colsample_bynode=0.5)
xgb.fit(X_train, y_train)
xgb_predicted = xgb.predict(X_test)
xgb_conf_matrix = confusion_matrix(y_test, xgb_predicted)
xgb_acc_score = accuracy_score(y_test, xgb_predicted)
print("confussion matrix")
print(xgb_conf_matrix)
print("\n")
print("Accuracy of Extreme Gradient Boost:",xgb_acc_score*100,'\n')
print(classification_report(y_test,xgb_predicted))

confussion matrix
[[250   0]
 [  0 236]]


Accuracy of Extreme Gradient Boost: 100.0 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       250
           1       1.00      1.00      1.00       236

    accuracy                           1.00       486
   macro avg       1.00      1.00      1.00       486
weighted avg       1.00      1.00      1.00       486

